In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mp
import sys, os, glob

Get list of available samples

In [2]:
sample_names = sorted(glob.glob("../03_samples/*/*.csv"))

Define testing set name

In [3]:
testing = '20190717-122651'

Determine samples that are successive (target last 3 to predict next)

In [4]:
idx_train = []
for i in range(0,len(sample_names)-3):
    date_name1 = sample_names[i+0][-24:-9]
    date_name2 = sample_names[i+1][-24:-9]
    date_name3 = sample_names[i+2][-24:-9]
    date_name4 = sample_names[i+3][-24:-9]
    if date_name1 == date_name2 and date_name2 == date_name3 and date_name3 == date_name4 and date_name1 != testing:
        sample_num1 = int(sample_names[i+0][-8:-4])
        sample_num2 = int(sample_names[i+1][-8:-4])
        sample_num3 = int(sample_names[i+2][-8:-4])
        sample_num4 = int(sample_names[i+3][-8:-4])
        if sample_num1+1 == sample_num2 and sample_num2+1 == sample_num3 and sample_num3+1 == sample_num4:
            idx_train.append(i)
            
idx_test = []
for i in range(0,len(sample_names)-3):
    date_name1 = sample_names[i+0][-24:-9]
    date_name2 = sample_names[i+1][-24:-9]
    date_name3 = sample_names[i+2][-24:-9]
    date_name4 = sample_names[i+3][-24:-9]
    if date_name1 == date_name2 and date_name2 == date_name3 and date_name3 == date_name4 and date_name1 == testing:
        sample_num1 = int(sample_names[i+0][-8:-4])
        sample_num2 = int(sample_names[i+1][-8:-4])
        sample_num3 = int(sample_names[i+2][-8:-4])
        sample_num4 = int(sample_names[i+3][-8:-4])
        if sample_num1+1 == sample_num2 and sample_num2+1 == sample_num3 and sample_num3+1 == sample_num4:
            idx_test.append(i)    

global parameters

In [5]:
n_samples_max = 350
train_batch_size = len(idx_train)
test_batch_size = len(idx_test)

Init empty array for features and target

In [6]:
X_train = np.zeros((3*n_samples_max,train_batch_size),dtype=int)
Y_train = np.zeros((n_samples_max,train_batch_size),dtype=int)
X_test = np.zeros((3*n_samples_max,test_batch_size),dtype=int)
Y_test = np.zeros((n_samples_max,test_batch_size),dtype=int)

Fill arrays with data

In [7]:
len_X_train = 0
len_Y_train = 0
len_X_test = 0
len_Y_test = 0
for i in range(0,len(idx_train)):
    sample1 = pd.read_csv(sample_names[idx_train[i]+0],header=None,usecols=[1]).values.T[0]
    sample2 = pd.read_csv(sample_names[idx_train[i]+1],header=None,usecols=[1]).values.T[0]
    sample3 = pd.read_csv(sample_names[idx_train[i]+2],header=None,usecols=[1]).values.T[0]
    sample4 = pd.read_csv(sample_names[idx_train[i]+3],header=None,usecols=[1]).values.T[0]
    X_train[0:len(sample1),i] = sample1
    X_train[len(sample1):len(sample1)+len(sample2),i] = sample2
    X_train[len(sample1)+len(sample2):len(sample1)+len(sample2)+len(sample3),i] = sample3
    len_X_train = max(len_X_train,len(sample1)+len(sample2)+len(sample3))
    Y_train[0:len(sample4),i] = sample4
    len_Y_train = max(len_Y_train,len(sample4))
    
for i in range(0,len(idx_test)):
    sample1 = pd.read_csv(sample_names[idx_test[i]+0],header=None,usecols=[1]).values.T[0]
    sample2 = pd.read_csv(sample_names[idx_test[i]+1],header=None,usecols=[1]).values.T[0]
    sample3 = pd.read_csv(sample_names[idx_test[i]+2],header=None,usecols=[1]).values.T[0]
    sample4 = pd.read_csv(sample_names[idx_test[i]+3],header=None,usecols=[1]).values.T[0]
    X_test[0:len(sample1),i] = sample1
    X_test[len(sample1):len(sample1)+len(sample2),i] = sample2
    X_test[len(sample1)+len(sample2):len(sample1)+len(sample2)+len(sample3),i] = sample3
    len_X_test = max(len_X_test,len(sample1)+len(sample2)+len(sample3))
    Y_test[0:len(sample4),i] = sample4
    len_Y_test = max(len_Y_test,len(sample4))

Take sample length from both training and testing sets for both

In [8]:
len_X = max(len_X_test,len_X_train)
len_Y = max(len_Y_test,len_Y_train)

Save arrays to csv

In [9]:
np.savetxt('X_train.csv', X_train[0:len_X,:], delimiter=',',fmt='%d')
np.savetxt('Y_train.csv', Y_train[0:len_Y,:], delimiter=',',fmt='%d')
np.savetxt('X_test.csv', X_test[0:len_X,:], delimiter=',',fmt='%d')
np.savetxt('Y_test.csv', Y_test[0:len_Y,:], delimiter=',',fmt='%d')